In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as pli
import seaborn as sns
import os
import tensorflow as tf
from PIL import Image
import torch
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import classification_report

In [2]:
def clean_colab_memory(path):
    !rm -r {path}

def save_to_drive(src, dest):
    !cp -r {src} {dest}

# Esempio di utilizzo
# clean_colab_memory('/content/nih_chest_xray_subset')
# save_to_drive('/content/nih_chest_xray_subset', '/content/drive/My Drive/')


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd

labels_path = "/content/drive/MyDrive/nih_chest_xray_subset/sample_labels.csv"
labels_df = pd.read_csv(labels_path)

In [5]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5606 entries, 0 to 5605
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Image Index                  5606 non-null   object 
 1   Finding Labels               5606 non-null   object 
 2   Follow-up #                  5606 non-null   int64  
 3   Patient ID                   5606 non-null   int64  
 4   Patient Age                  5606 non-null   object 
 5   Patient Gender               5606 non-null   object 
 6   View Position                5606 non-null   object 
 7   OriginalImageWidth           5606 non-null   int64  
 8   OriginalImageHeight          5606 non-null   int64  
 9   OriginalImagePixelSpacing_x  5606 non-null   float64
 10  OriginalImagePixelSpacing_y  5606 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 481.9+ KB


In [6]:
labels_df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,0.139
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M,AP,2500,2048,0.168,0.168
2,00000017_001.png,No Finding,1,17,077Y,M,AP,2500,2048,0.168,0.168
3,00000030_001.png,Atelectasis,1,30,079Y,M,PA,2992,2991,0.143,0.143
4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F,AP,2500,2048,0.168,0.168


In [7]:
labels_df.shape

(5606, 11)

In [8]:
labels_df['Finding Labels'].value_counts()

,count
Finding Labels,
No Finding,3044
Infiltration,503
Effusion,203
Atelectasis,192
Nodule,144
...,...
Atelectasis|Edema|Effusion|Infiltration|Pneumonia,1
Atelectasis|Consolidation|Edema|Infiltration|Pneumonia,1
Atelectasis|Effusion|Hernia,1


In [9]:
data = []

for _, row in labels_df.iterrows():
    image_path = os.path.join('/content/drive/MyDrive/nih_chest_xray_subset/images', row['Image Index'])
    labels = row['Finding Labels'].split('|')
    data.append({'image_path': image_path, 'labels': labels})

print(data[:5])

[{'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000013_005.png', 'labels': ['Emphysema', 'Infiltration', 'Pleural_Thickening', 'Pneumothorax']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000013_026.png', 'labels': ['Cardiomegaly', 'Emphysema']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000017_001.png', 'labels': ['No Finding']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000030_001.png', 'labels': ['Atelectasis']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000032_001.png', 'labels': ['Cardiomegaly', 'Edema', 'Effusion']}]


In [10]:
sum(["Infiltration" in r["labels"] for r in data])

967

In [11]:
sum(["Infiltration" not in r["labels"] for r in data])

4639

In [12]:
def preprocess_item(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [224, 224]) / 255.0
    return image, label

paths = [item['image_path'] for item in data]
labels = [1 if "Infiltration" in item["labels"] else 0 for item in data]

# Preprocessing con parallelismo
dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
dataset = dataset.map(lambda x, y: preprocess_item(x, y), num_parallel_calls=tf.data.AUTOTUNE)

dataset = dataset.cache()

# Shuffle per migliorare il training
dataset = dataset.shuffle(buffer_size=1000)  # Cambia il buffer in base al dataset

# Creazione dei batch # Prefetch per caricare il batch successivo in parallelo
dataset = dataset.batch(64).prefetch(tf.data.AUTOTUNE)

In [13]:
for images, labels in dataset.take(1):
    print(f"Batch shape: {images.shape}, Labels shape: {labels.shape}")

Batch shape: (32, 224, 224, 3), Labels shape: (32,)


In [14]:
loaded_dataset = tf.data.Dataset.load("/content/drive/MyDrive/nih_chest_xray_subset/processed_dataset")

In [15]:
#total_images = 0
#for images, labels in dataset:
#   total_images += images.shape[0]
#
#print(f"Total_number_of: {total_images}")

In [16]:
# tf.data.experimental.save(dataset,"/content/drive/MyDrive/nih_chest_xray_subset/processed_dataset")

# TensorFlow Keras

In [17]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False # don't update base_model weights during training

model = tf.keras.Sequential([base_model, tf.keras.layers.GlobalAveragePooling2D(), tf.keras.layers.Dense(1, activation='sigmoid')])

In [18]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='binary_crossentropy',metrics=['accuracy'])

In [26]:
dataset_size = len(dataset)

test_size = int(dataset_size * 0.2)
validation_size = int(dataset_size * 0.1)
train_size = dataset_size - test_size - validation_size

test_set = dataset.take(test_size)  # Primi 20% per il test
validation_set = dataset.skip(test_size).take(validation_size)  # Successivi 20% per validazione
train_set = dataset.skip(test_size + validation_size)  # Resto per il training


In [27]:
print(f"Dataset totale (batch): {dataset_size}")
print(f"Test set (batch): {test_size}")
print(f"Validation set (batch): {validation_size}")
print(f"Training set (batch): {train_size}")

Dataset totale (batch): 176
Test set (batch): 35
Validation set (batch): 17
Training set (batch): 124


In [28]:
history = model.fit(train_set, epochs=5, validation_data = validation_set)

Epoch 1/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 603s 4s/step - accuracy: 0.8134 - loss: 0.4652 - val_accuracy: 0.8548 - val_loss: 0.3844
Epoch 2/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 195s 2s/step - accuracy: 0.8262 - loss: 0.4422 - val_accuracy: 0.8585 - val_loss: 0.3934
Epoch 3/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 223s 2s/step - accuracy: 0.8313 - loss: 0.4320 - val_accuracy: 0.8548 - val_loss: 0.4075
Epoch 4/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step - accuracy: 0.8376 - loss: 0.4335 - val_accuracy: 0.8382 - val_loss: 0.4120
Epoch 5/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 195s 2s/step - accuracy: 0.8278 - loss: 0.4300 - val_accuracy: 0.8787 - val_loss: 0.3711


In [29]:
test_loss, test_accuracy = model.evaluate(test_set)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

35/35 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.8690 - loss: 0.3828
Test Loss: 0.38470759987831116, Test Accuracy: 0.862500011920929


In [30]:
y_true = []  # Ground truth
y_pred = []  # Predizioni
for images, labels in test_set:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend((preds > 0.5).astype(int).flatten())

print(classification_report(y_true, y_pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 

This is due to the imbalance of the dataset: the model almost always predicts 0, largely ignoring class 1.  
On class 1 we experienced:  
Precision = 50%: The model makes few predictions like 1, but half of these predictions are correct.  
Recall = 2%: The model identifies only 2% of the real class 1 examples

# Pytorch

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class CovNet(nn.Module):
  def __init__(self, channels, classes):
    super(CovNet, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=channels, out_channels=32, kernel_size=(5,5), padding='same')
    self.relu1 = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))

    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(5,5), padding='same')
    self.relu2 = nn.ReLU()
    self.maxpool2 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))

    self.flatten = nn.Flatten()

    self.fc1 = nn.Linear(in_features=64*56*56, out_features=128)
    self.relu3 = nn.ReLU()

    self.fc2 = nn.Linear(in_features=128, out_features=classes)
    self.softmax = nn.Softmax(dim=1)

  def forward_pass(self, x):
    x = nn.functional.relu(self.conv1(x))
    x = self.maxpool1(x)
    x = nn.functional.relu(self.conv2(x))
    x = self.maxpool2
    x = self.flatten(x)
    x = nn.functional.relu(self.fc1(x))
    x = self.fc2(x)
    x = self.softmax(x)
    return x

In [ ]:
num_classes = 2
num_epochs = 10
batch_size = 32
learning_rate = 0.001

In [ ]:
model = CovNet(input,channels=3, classes=num_classes).to(device)

In [ ]:
train_dataset = #Dataset
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = #Dataset
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
history = {
    'train_loss': [],
    'train_acc': [],
    'test_loss': [],
    'test_acc': []
}

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    for batch_size, (data, targets) in enumerate(tqdm(train_dataset)):

        data = data.to(device)
        targets = targets.to(device)

        scores = model(data)
        loss = loss(scores, targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()